# Allergan - List
## Next Year
* Sieht jedes Jahr anders aus. Alte Scripts prüfen

In [82]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *
import re

## Import PDF

In [83]:
# Read pdf into DataFrame
columns= [132, 244, 408, 458, 519, 567, 624, 675, 757]
df = tabula.read_pdf("pkk_2019_datenerfassung_schweiz.pdf", pages='all', lattice=False, columns=columns, guess=False)

## Format Table

In [84]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['name', 'empty1', 'address', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total']

#Add Type
index_hco = df_export[df_export['name'].str.contains("AGGREGIERTE OFFENLEGUNG", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove column empty
df_export = df_export.drop(columns=['empty1'])

#Replace"nicht anwendbar"
df_export['donations_grants'] = df_export['donations_grants'].str.replace('nicht anwendbar', '', regex=False)
df_export['sponsorship'] = df_export['sponsorship'].str.replace('nicht anwendbar', '', regex=False)

#Remove empty
df_export = df_export.dropna(subset=['address'], how='all')
df_export = df_export.dropna(subset=['total'], how='all')
df_export = df_export.dropna(subset=['name'], how='all')

#Remove by String Comparision
df_export = df_export[~df_export.name.str.contains('Prozentualer Anteil', na=False)]

#Remove -
df_export = remove_in_numbers(df_export, '-')

#Clean Number
df_export = cleanup_number(df_export)

#Convert to Numbers
df_export = amounts_to_number(df_export)

#Remove Carination
df_export = remove_carination(df_export)

#Add fields
df_export = add_plz(df_export)
df_export = add_country(df_export)
df_export = add_uci(df_export)

def copy_plz(value):
    #Only check Strings
    if isinstance(value, float):
        return np.NaN
    
    r = re.search('\s?[\d]{4}\s?', value)
    if r:
        return r.group(0).strip()
    else:
        return np.NaN

#Copy plz in location
df_export['location'] = df_export['address'].apply(lambda x: x.split(',')[len(x.split(',')) - 1])
df_export['address'] = df_export.apply(lambda row: row['address'].replace(',' + row['location'], ''), axis=1)
    
#Copy into PLZ
df_export.loc[df_export.plz.isna(), 'plz'] = df_export.loc[df_export.plz.isna(), 'location'].apply(copy_plz)

#Clean
df_export.loc[df_export.plz.notna(), 'location'] = df_export['location'].str.replace('\s?[\d]{4}\s?', '', regex=True)

#Strip
df_export['location'] = df_export['location'].str.strip()
df_export['address'] = df_export['address'].str.strip()

#basic string conversion
df_export = basic_string_conversion(df_export)

#Export
export_list(df_export, 'allergan')

#df_export.head()
write_to_excel(df_export, 'tmp.xlsx', open=True)

saved


In [ ]:
#write_to_excel(df_export, 'tmp.xlsx', open=True)